In [1]:
import pandas as pd
import time
import numpy as np
import xlsxwriter

In [47]:
f = pd.ExcelFile("data_more.xlsx")

In [48]:
f.sheet_names

['BSE_Sensex', 'NIFTY_50', 'Infosys', 'Reliance']

In [49]:
BSE_Sensex = f.parse('BSE_Sensex', index_col='Date')
NIFTY_50 = f.parse('NIFTY_50', index_col='Date')
Reliance = f.parse('Reliance', index_col='Date')
Infosys = f.parse('Infosys', index_col='Date')


# Define stock indicator functions

In [50]:
def moving_average(df, n):
    """Calculate the moving average for the given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))
    df = df.join(MA)
    return df


In [51]:
def weighted_moving_average(df, n):
    """Calculate the weighted moving average for the given data.
    """
    weighted_sum = n * df['Close'].copy()
    for i in range(1, 10):
        weighted_sum += (n-i) * df['Close'].shift(i)
    weighted_sum /= sum(range(n))
    
    WMA = pd.Series(weighted_sum, name='WMA_' + str(n))
    df = df.join(WMA)
    return df


In [52]:
def momentum(df, n):
    '''
    It measures the amount that a security’s price has changed over a given time span n.
    '''
    M = pd.Series(df['Close'].diff(n), name='Momentum_' + str(n))
    df = df.join(M)
    return df

In [53]:
def stochastic_oscillator_k(df, n):
    """Calculate stochastic oscillator %K for given data.
    
    Stochastic %K. It compares where a security’s price closed relative to its price range over a given time period.

    """
    SOk = pd.Series((df['Close'] - df['Low'].rolling(n).min()) / \
                    (df['High'].rolling(n).max() - df['Low'].rolling(n).min()), name='SO%k')
    df = df.join(SOk)
    return df


def stochastic_oscillator_d(df, n):
    """Calculate stochastic oscillator %D for given data.
    
    Stochastic slow %D. Moving average of %K.
    """
    
    SOd = pd.Series(df['SO%k'].rolling(n).mean(), name='SO%d_' + str(n))
    df = df.join(SOd)
    return df

In [54]:
def relative_strength_index(df, n):
    """Calculate Relative Strength Index(RSI) for given data.

    """
    
    up = pd.Series([x if x > 0 else 0 for x in df['Close'] - df['Close'].shift(1)])
    down = pd.Series([x if x < 0 else 0 for x in df['Close'] - df['Close'].shift(1)])

    RSI = pd.Series((100 - 100/(1 + (up.rolling(n).mean() / down.rolling(n).mean()))).values, index=df.index,name='RSI_' + str(n))
    df = df.join(RSI)
    return df


In [55]:
def macd(df, n_fast, n_slow, n_MACD):
    """Calculate MACD, MACD Signal and MACD difference
    EMA: https://towardsdatascience.com/trading-toolbox-02-wma-ema-62c22205e2a9
    MACD: https://school.stockcharts.com/doku.php?id=technical_indicators:moving_average_convergence_divergence_macd
    
    
    """
    EMAfast = pd.Series(df['Close'].ewm(span=n_fast, min_periods=n_slow).mean())
    EMAslow = pd.Series(df['Close'].ewm(span=n_slow, min_periods=n_slow).mean())
    MACD = pd.Series(EMAfast - EMAslow, name='MACD_' + str(n_fast) + '_' + str(n_slow))
    MACDsign = pd.Series(MACD.ewm(span=n_MACD, min_periods=n_MACD).mean(), name='MACD')
    #MACDsign = pd.Series(MACD.ewm(span=n_MACD, min_periods=n_MACD).mean(), name='MACDsign_' + str(n_fast) + '_' + str(n_slow))
    #MACDdiff = pd.Series(MACD - MACDsign, name='MACDdiff_' + str(n_fast) + '_' + str(n_slow))
    #df = df.join(MACD)
    df = df.join(MACDsign)
    #df = df.join(MACDdiff)
    return df

In [56]:
def Larry_William_R(df, n):
    """Calculate Larry William’s R% for given data.
    
    Williams %R, also known as the Williams Percent Range, is a type of momentum indicator that moves between 0 and -100 
    and measures overbought and oversold levels. 
    
    https://school.stockcharts.com/doku.php?id=technical_indicators:williams_r
    
    """
    R = pd.Series(-100*(df['High'].rolling(n).max() - df['Close']) /\
                  (df['High'].rolling(n).max() - df['Low'].rolling(n).min()), name='LW_R')
    df = df.join(R)
    return df


In [57]:
def AD_oscillator(df):
    """Calculate AD_oscillator for given data.
    """
    AD = pd.Series((df['High'] - df['Close'].shift(1)) / (df['High'] - df['Low']), name='AD_R')
    df = df.join(AD)
    return df



In [58]:
def commodity_channel_index(df, n):
    """Calculate Commodity Channel Index for given data.
    
    """
    PP = (df['High'] + df['Low'] + df['Close']) / 3
    CCI = pd.Series((PP - PP.rolling(n, min_periods=n).mean()) / (0.015*PP.rolling(n).apply(lambda x: x.mad())),
                    name='CCI_' + str(n))
    df = df.join(CCI)
    return df

# Process all indicator features

In [99]:

with pd.ExcelWriter('data_feature_more.xlsx') as writer:
    for security_name in f.sheet_names:
        data = f.parse(security_name, index_col='Date')
        data = moving_average(data, 10)
        data = weighted_moving_average(data, 10)
        data = momentum(data, 10)
        data = stochastic_oscillator_k(data, 10)
        data = stochastic_oscillator_d(data, 10)
        data = relative_strength_index(data, 10)
        data = macd(data,12,26,10)
        data = Larry_William_R(data, 10)
        data = AD_oscillator(data)
        data = commodity_channel_index(data, 10)
        
        data['label'] = (data['Close']-data['Close'].shift(1)).\
                        apply(lambda x: '1' if x > 0 else '0' if x <= 0 else np.nan)
        
        data_feature = data.iloc[:,3: ].copy()
        # use previous day value to predict future
        data_feature.iloc[:, 1:-1] = data_feature.iloc[:, 1:-1].shift(1)

        data_feature = data_feature.dropna()
        for col in data_feature.columns[:-1]:
            data_feature[col] = data_feature[col].replace(float('inf'), float('Nan'))
            data_feature[col] = data_feature[col].replace(float('-inf'), float('Nan'))
            data_feature[col] = data_feature[col].replace(float('Nan'), data_feature[col].mean())            
        
        data_feature.to_excel(writer, sheet_name = security_name)


### Some sample data sanity checks

In [210]:
BSE_Sensex = moving_average(BSE_Sensex, 10)

BSE_Sensex = weighted_moving_average(BSE_Sensex, 10)
BSE_Sensex = momentum(BSE_Sensex, 10)
BSE_Sensex = stochastic_oscillator_k(BSE_Sensex, 10)
BSE_Sensex = stochastic_oscillator_d(BSE_Sensex, 10)
BSE_Sensex = relative_strength_index(BSE_Sensex, 10)
BSE_Sensex = macd(BSE_Sensex,12,26,10)
BSE_Sensex = Larry_William_R(BSE_Sensex, 10)
BSE_Sensex = AD_oscillator(BSE_Sensex)
BSE_Sensex = commodity_channel_index(BSE_Sensex, 10)


In [214]:
BSE_Sensex['label'] = (BSE_Sensex['Close']-BSE_Sensex['Close'].shift(1)).\
                        apply(lambda x: '1' if x > 0 else '0' if x <= 0 else np.nan)

In [215]:
BSE_Sensex

,Open,High,Low,Close,MA_10,WMA_10,Momentum_10,SO%k,SO%d_10,RSI_10,MACD,LW_R,AD_R,CCI_10,label
Date,,,,,,,,,,,,,,,
2003-01-01,3383.85,3396.80,3381.07,3390.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-01-02,3399.72,3407.24,3363.11,3365.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.387945,NaN,0
2003-01-03,3380.44,3386.97,3351.89,3357.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.624572,NaN,0
2003-01-06,3364.72,3364.72,3330.02,3334.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.206916,NaN,0
2003-01-07,3350.18,3352.43,3320.42,3330.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547954,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-24,19278.00,19347.64,19237.26,19255.09,19332.509,23625.717778,-154.60,0.228997,0.423104,NaN,173.401677,-77.100291,0.957057,-60.229015,1
2012-12-26,19302.47,19468.40,19274.07,19417.46,19335.541,23644.595778,30.32,0.722596,0.417966,NaN,168.423581,-27.740390,1.097669,49.492220,1
2012-12-27,19479.84,19504.40,19302.21,19323.80,19332.395,23641.986667,-31.46,0.470469,0.399800,NaN,162.481554,-52.953053,0.429992,40.577801,0


In [216]:
BSE_Sensex_feature = BSE_Sensex.iloc[:,4: ]


In [218]:
BSE_Sensex_feature[:20]

,MA_10,WMA_10,Momentum_10,SO%k,SO%d_10,RSI_10,MACD,LW_R,AD_R,CCI_10,label
Date,,,,,,,,,,,
2003-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.387945,NaN,0
2003-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.624572,NaN,0
2003-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.206916,NaN,0
2003-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547954,NaN,0
2003-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.364107,NaN,1
2003-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.383843,NaN,1
2003-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.437433,NaN,0
2003-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.063035,NaN,0


In [220]:
f.sheet_names

['BSE_Sensex', 'NIFTY_50', 'Reliance', 'Infosys']